In [1]:
import config
import requests
from pandas import json_normalize
import json
import time
import csv


ModuleNotFoundError: No module named 'pandas.json_normalize'

## Authenticate with Strava

In [ ]:
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)

if strava_tokens['expires_at'] < time.time():
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': config.client_id,
                                'client_secret': config.client_secret,
                                'grant_type': 'refresh_token',
                                'refresh_token': strava_tokens['refresh_token']
                                }
                    )
    new_strava_tokens = response.json()

    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)

    strava_tokens = new_strava_tokens

In [ ]:
with open('strava_tokens.json') as check:
  data = json.load(check)
print(data)

{'token_type': 'Bearer', 'expires_at': 1635097218, 'expires_in': 20102, 'refresh_token': 'f59c3f6a75c357f9754f213cad01a09b1d6e41d0', 'access_token': 'de53662bafe54e394d68f9280af88781088b9651', 'athlete': {'id': 17590055, 'username': None, 'resource_state': 2, 'firstname': 'Conan', 'lastname': 'Mercer', 'bio': '', 'city': 'Girona', 'state': 'Catalonia', 'country': 'Spain', 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2016-09-17T12:32:14Z', 'updated_at': '2021-10-23T16:54:36Z', 'badge_type_id': 0, 'weight': 78.0178, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/17590055/13297565/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/17590055/13297565/1/large.jpg', 'friend': None, 'follower': None}}


In [ ]:
# Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
# Loop through all activities
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
# Get first page of activities from Strava with all fields
r = requests.get(url + '?access_token=' + access_token)
r = r.json()
    
df = json_normalize(r)
df.to_csv('strava_activities_all_fields.csv')

NameError: name 'json_normalize' is not defined